In [82]:
import numpy as np
import cv2 as cv

img_sol1 = cv.imread("data/lena.png")

## solution 1
point1 = (60, 40)
point2 = (420, 510)

img_hsv = cv.cvtColor(img_sol1, cv.COLOR_BGR2HSV)
img_hsv[..., -1] = cv.equalizeHist(img_hsv[..., -1])
img_hsv_equal = cv.cvtColor(img_hsv, cv.COLOR_HSV2BGR)


cv.rectangle(img_hsv_equal, point1, point2, (0, 0, 255), 3)

img_hflip = img_hsv_equal[:,::-1,:]

img_resize = cv.resize(img_hflip,None, fx=0.5, fy=0.5)


while True:
    
    cv.imshow('image',img_resize)
    k = cv.waitKey(0)
    if k == 27:
        cv.destroyAllWindows()
        break

In [83]:
img_sol2 = img_sol1.copy()


img_hsv = cv.cvtColor(img_sol2, cv.COLOR_BGR2HSV)
img_hsv[..., -1] = cv.equalizeHist(img_hsv[..., -1])
img_hsv_equal = cv.cvtColor(img_hsv, cv.COLOR_HSV2BGR)
img_hflip = img_hsv_equal[:,::-1,:]

# 透過四則運算計算鏡像後位置
# 確保點的位置一樣是左上跟右下，所以交換鏡像後的 x 座標 (y 座標做水平鏡像後位置不變)
point1 = (420, 40)
point2 = (60, 510)

"""
縮放處理 (0.5 倍)
"""
fx = 0.5
fy = 0.5
resize_col = int(img_hflip.shape[1]*fx)
resize_row = int(img_hflip.shape[0]*fy)

# 建構 scale matrix
M_Scale = np.array([[fx,0,0],
                   [0,fy,0]],dtype=np.float32)
img_final = cv.warpAffine(img_hflip,M_Scale,(resize_col,resize_row))

# 把左上跟右下轉為矩陣型式
bbox = np.array((point1, point2), dtype=np.float32)
#print('M_scale.shape={}, bbox.shape={}'.format(M_scale.shape, bbox.shape))

# 做矩陣乘法可以使用 `np.dot`, 為了做矩陣乘法, M_scale 需要做轉置之後才能相乘
homo_coor_result = np.dot(M_scale.T, bbox)
homo_coor_result = homo_coor_result.astype('uint8')
#print(homo_coor_result)
scale_point1 = tuple(homo_coor_result[0])
scale_point2 = tuple(homo_coor_result[1])
print('origin point1={}, origin point2={}'.format(point1, point2))
print('resize point1={}, resize point2={}'.format(scale_point1, scale_point2))

# 畫圖
cv.rectangle(img_final, scale_point1, scale_point2, (0, 0, 255), 3)

while True:
    cv.imshow('image', img_final)
    k = cv.waitKey(0)
    if k == 27:
        cv.destroyAllWindows()
        break

origin point1=(420, 40), origin point2=(60, 510)
resize point1=(210, 20), resize point2=(30, 255)
